# LLM Optimization Modelling Experiment

In [30]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel
from IPython.display import Markdown

## 1. Define the problem description

In [128]:
problem = '''A firm that packs refreshments and beers, situated in the province of Valencia (Spain) employs the same syrup to produce its 1.5 l COLI and PEPSA products on its S1 production line. Once processed, each hectolitre of syrup produces 40 units of the 1.5 l COLI product and 20 units of the 1.5 l PEPSA product. If X1 units of the 1.5 l COLI product and X2 units of the 1.5 l PEPSA product are produced, the firm estimates that the daily income obtained in dollars would be given by the following function:
49000 times X1 minus X1 squared plus 30 times X2 minus two times X2 squared. 
It costs 150 dollars to buy and process each hectolitre of syrup. The S1 packaging line has a net capacity of producing 7100 1.5 l product units every hour. The firm works 5 days a week in 8h shifts. Given its weekly target coverage, the firm is committed to produce at least half the amount of PEPSA than COLI. Although priority orders tend to amend its production planning, the firm wishes to have a basic product planning that optimises its daily profits. 
'''

## 2. Generate the mathematical model

In [137]:
#Initializing the session. To replicate, make sure the right credentials are saved in a PATH variable
PROJECT_ID = "llm4optproblems"
REGION = "us-central1"
vertexai.init(project=PROJECT_ID, location=REGION)

#Specifying the model
generative_multimodal_model = GenerativeModel("gemini-1.5-pro-preview-0409")

#The propmt applied to all problems
prompt = '''Let's think step by step. Please write a mathematical optimization model for this problem. If there are parameter values, make sure to include them in the mathematical formulation.
'''

#Generate the response
response = generative_multimodal_model.generate_content([prompt+problem])


In [138]:
#Show the resopnse in a formatted way
Markdown(response.text)

## Mathematical Optimization Model for Production Planning

**Objective Function (Maximize Daily Profit):**

```
Maximize Z = 49000X1 - X1^2 + 30X2 - 2X2^2 - 150H
```

Where:

* Z: Daily profit in dollars
* X1: Number of 1.5L COLI units produced daily
* X2: Number of 1.5L PEPSA units produced daily
* H: Number of hectoliters of syrup used daily 

**Constraints:**

1. **Syrup Usage:** The amount of syrup used should be sufficient to produce both products.
    * `40X1 + 20X2 <= 100H` (since each hectoliter produces 40 COLI and 20 PEPSA units)

2. **Production Line Capacity:** The total production cannot exceed the line's capacity.
    * `X1 + X2 <= 7100 * 8 * 5` (considering 8-hour shifts and 5 working days)

3. **Minimum PEPSA Production:**  PEPSA production should be at least half of COLI production. 
    * `X2 >= 0.5 * X1`

4. **Non-negativity:** Production quantities cannot be negative.
    * `X1 >= 0`
    * `X2 >= 0`
    * `H >= 0`

**Parameters:**

* Syrup cost per hectoliter: $150
* Production line capacity: 7100 units per hour
* Working days per week: 5
* Shift duration: 8 hours per day
* COLI production per hectoliter of syrup: 40 units
* PEPSA production per hectoliter of syrup: 20 units

**Solving the Model:**

This model is a non-linear optimization problem due to the squared terms in the objective function. Various software tools and solvers can be used to find the optimal values of X1, X2, and H that maximize the daily profit Z while satisfying all constraints. 


## 3. Generate the pyomo code

In [139]:
#Second prompt gets the output of the previous step and generates the code
prompt2 = "Please write pyomo code for this mathematical problem. Use sample data where needed. Indicate where you use sample data. \n"
prompt2 += response.text 
response2 = generative_multimodal_model.generate_content([prompt2])

In [140]:
#Showing the code in a formatted way
Markdown(response2.text)

## Pyomo Implementation 

```python
from pyomo.environ import *

# Model creation
model = ConcreteModel(name="ProductionPlanning")

# Decision variables
model.X1 = Var(domain=NonNegativeReals, doc="Number of 1.5L COLI units produced daily")
model.X2 = Var(domain=NonNegativeReals, doc="Number of 1.5L PEPSA units produced daily")
model.H = Var(domain=NonNegativeReals, doc="Number of hectoliters of syrup used daily")

# Objective function
model.profit = Objective(
    expr=49000 * model.X1 - model.X1**2 + 30 * model.X2 - 2 * model.X2**2 - 150 * model.H,
    sense=maximize,
    doc="Maximize daily profit"
)

# Constraints
model.syrup_usage = Constraint(expr=40 * model.X1 + 20 * model.X2 <= 100 * model.H)
model.production_capacity = Constraint(expr=model.X1 + model.X2 <= 7100 * 8 * 5) 
model.min_pepsa_production = Constraint(expr=model.X2 >= 0.5 * model.X1)

# Note on sample data:
# The parameters provided in the problem description (syrup cost, production capacity, etc.)
# are directly used in the constraints and objective function as numerical values.
```

**Explanation:**

* We import the `pyomo.environ` module to use Pyomo's modeling objects.
* We create a concrete model instance named `ProductionPlanning`.
* We define the decision variables `X1`, `X2`, and `H` with their respective domains (`NonNegativeReals`). 
* The objective function is defined using the `Objective` object, specifying the expression to maximize and the sense of optimization (`maximize`).
* Each constraint is defined using the `Constraint` object with its corresponding expression.
* We directly use the provided parameters (like syrup cost, production capacity) within the constraints and objective function. 

**Solving the Model:**

To solve the model, you would typically choose a solver like ipopt or cplex and use Pyomo's `SolverFactory` to create a solver instance and solve the model. The specific steps for solving may vary depending on the chosen solver and its interface.


## 4. Input problem data and try running the generated code

In [142]:
from pyomo.environ import *

# Model creation
model = ConcreteModel(name="ProductionPlanning")

# Decision variables
model.X1 = Var(domain=NonNegativeReals, doc="Number of 1.5L COLI units produced daily")
model.X2 = Var(domain=NonNegativeReals, doc="Number of 1.5L PEPSA units produced daily")
model.H = Var(domain=NonNegativeReals, doc="Number of hectoliters of syrup used daily")

# Objective function
model.profit = Objective(
    expr=49000 * model.X1 - model.X1**2 + 30 * model.X2 - 2 * model.X2**2 - 150 * model.H,
    sense=maximize,
    doc="Maximize daily profit"
)

# Constraints
model.syrup_usage = Constraint(expr=40 * model.X1 + 20 * model.X2 <= 100 * model.H)
model.production_capacity = Constraint(expr=model.X1 + model.X2 <= 7100 * 8 * 5) 
model.min_pepsa_production = Constraint(expr=model.X2 >= 0.5 * model.X1)

# Solve the model
solver = SolverFactory('ipopt')
solver.solve(model)

# Print the results
print("Optimal Production Quantities:")

print(f"Maximum Profit: {model.profit():.2f}")

Optimal Production Quantities:
Maximum Profit: 399187266.67


## 5. Correct the code to verify model viability (optional)

## 6. Printing the outputs as strings, so they can be saved.
Those can be rendered as markdown for better readability

In [143]:
print(response.text)

## Mathematical Optimization Model for Production Planning

**Objective Function (Maximize Daily Profit):**

```
Maximize Z = 49000X1 - X1^2 + 30X2 - 2X2^2 - 150H
```

Where:

* Z: Daily profit in dollars
* X1: Number of 1.5L COLI units produced daily
* X2: Number of 1.5L PEPSA units produced daily
* H: Number of hectoliters of syrup used daily 

**Constraints:**

1. **Syrup Usage:** The amount of syrup used should be sufficient to produce both products.
    * `40X1 + 20X2 <= 100H` (since each hectoliter produces 40 COLI and 20 PEPSA units)

2. **Production Line Capacity:** The total production cannot exceed the line's capacity.
    * `X1 + X2 <= 7100 * 8 * 5` (considering 8-hour shifts and 5 working days)

3. **Minimum PEPSA Production:**  PEPSA production should be at least half of COLI production. 
    * `X2 >= 0.5 * X1`

4. **Non-negativity:** Production quantities cannot be negative.
    * `X1 >= 0`
    * `X2 >= 0`
    * `H >= 0`

**Parameters:**

* Syrup cost per hectoliter: $150

In [144]:
print(response2.text)

## Pyomo Implementation 

```python
from pyomo.environ import *

# Model creation
model = ConcreteModel(name="ProductionPlanning")

# Decision variables
model.X1 = Var(domain=NonNegativeReals, doc="Number of 1.5L COLI units produced daily")
model.X2 = Var(domain=NonNegativeReals, doc="Number of 1.5L PEPSA units produced daily")
model.H = Var(domain=NonNegativeReals, doc="Number of hectoliters of syrup used daily")

# Objective function
model.profit = Objective(
    expr=49000 * model.X1 - model.X1**2 + 30 * model.X2 - 2 * model.X2**2 - 150 * model.H,
    sense=maximize,
    doc="Maximize daily profit"
)

# Constraints
model.syrup_usage = Constraint(expr=40 * model.X1 + 20 * model.X2 <= 100 * model.H)
model.production_capacity = Constraint(expr=model.X1 + model.X2 <= 7100 * 8 * 5) 
model.min_pepsa_production = Constraint(expr=model.X2 >= 0.5 * model.X1)

# Note on sample data:
# The parameters provided in the problem description (syrup cost, production capacity, etc.)
# are directly use